# Residential choice support system

03/05/2020 Yuki Fujimaru

## Abstract

Similarity of the cities in Fukuoka, in Japan, was analyzed and clustered in the study. I extracted the factors which affect the clustering result in additon to just clustering them.

## Table of contents

+ lattitude and longitude in each city of Fukuoka were derived from http://tide.jpn.org/ll/?l=%E7%A6%8F%E5%B2%A1%E7%9C%8C.
+ Venues data in Fukuoka were obtained from Foursquare API. 
+ Landscape data in Fukuoka including ratio of land use, such as rice field, industrial site, and rive, rwere obtained from https://www.pref.fukuoka.lg.jp/uploaded/attachment/109588.pdf.
+ Merged data of Venues data and Landscape data were constructed.

## Introductory section (Business understnding & Analytic approach)

Working from home remotely is increasing these days. According to the survey performed by IPSOS, 20 % of the labor population in the world work from home. This work-style change is bringing a new life-style that people easily move to any countries and cities without considering their jobs. In some cases, people have no home and continue changing their resident at a span. Considering these changes, we need a data platform and services for these people to search cities and area information. However, there are not enough services offering an instant search about feature of destrict you want to know. 
    
  The city data, such as venues, public transportation, landscape, and amusement facilities, are differently placed. For example, we can access venue data through Foursquare, but landscape data are stored at Ministry of Agriculture, Forestry, and Fishers, and public transportation data are placed in Japan Railway or other private railways. It is necessary to gather these data and make structured dataframe for acieving the system offering district information to people who consider living there.
 
 In this project, I will construct the model describing similarity among cities in Fukuoka, my home town and one of the biggest district in Japan, by k-means clustering. Moreover, I will visualize which featres contribute to a separation among clusters by Partial Least Square regression Discriminant Anlysis (PLS-DA).  

## Methodology section (Data requirement & Data collection & Data understanding & Data preparation)

To solve the adressed problem, the data including venues, landscape, public transportation in the cities in Fukuoka were collected.

### $Venues$ $data$

I got the data of venues in Fukuoka through Foursquare API by the following code

### Download latitude and longitude of cities in Fukuoka

In [1]:
import pandas as pd
url='http://tide.jpn.org/ll/?l=%E7%A6%8F%E5%B2%A1%E7%9C%8C'

df = pd.read_html(url,header=0)

In [2]:
df_latlog=df[0]
df_latlog2=df_latlog.drop(index=30).reset_index(drop=True)
df_latlog2

,市町村,緯度,経度,参考地点
0,北九州市門司区,33.885076,130.92079,青葉台
1,北九州市若松区,33.902525,130.687721,青葉台西一丁目
2,北九州市戸畑区,33.8977,130.824288,旭町
3,北九州市小倉北区,33.887916,130.867263,青葉一丁目
4,北九州市小倉南区,33.850402,130.907,安部山
...,...,...,...,...
66,京都郡苅田町,33.754071,130.975388,新津一丁目
67,京都郡みやこ町,33.678306,130.997909,呰見
68,築上郡吉富町,33.599343,131.168148,大字今吉
69,築上郡上毛町,33.574508,131.147191,大字安雲


### Call Venues data from Foursquare

In [3]:
def getNearbyVenues(names, latitudes, longitudes, radius=1500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [4]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

CLIENT_ID = '4XKVNDTMG0HKCUESIL3B0XS10R1ZPOMLNKHU2TZORV1GBAMT'
CLIENT_SECRET = 'VD3O0DHE4TXGG03CHQLNVLGTHVFLGKD2VF42DBI2X0FNITE3'
VERSION = '20180604'
LIMIT = 30

Fukuoka_venues=getNearbyVenues(names=df_latlog2['市町村'], latitudes=df_latlog2['緯度'].astype(np.float), longitudes=df_latlog2['経度'].astype(np.float))

北九州市門司区
北九州市若松区
北九州市戸畑区
北九州市小倉北区
北九州市小倉南区
北九州市八幡東区
北九州市八幡西区
福岡市東区
福岡市博多区
福岡市中央区
福岡市南区
福岡市西区
福岡市城南区
福岡市早良区
大牟田市
久留米市
直方市
飯塚市
田川市
柳川市
八女市
筑後市
大川市
行橋市
豊前市
中間市
小郡市
筑紫野市
春日市
大野城市
宗像市
太宰府市
古賀市
福津市
うきは市
宮若市
嘉麻市
朝倉市
みやま市
糸島市
筑紫郡那珂川町
糟屋郡宇美町
糟屋郡篠栗町
糟屋郡志免町
糟屋郡須恵町
糟屋郡新宮町
糟屋郡久山町
糟屋郡粕屋町
遠賀郡芦屋町
遠賀郡水巻町
遠賀郡岡垣町
遠賀郡遠賀町
鞍手郡小竹町
鞍手郡鞍手町
嘉穂郡桂川町
朝倉郡筑前町
朝倉郡東峰村
三井郡大刀洗町
三潴郡大木町
八女郡広川町
田川郡香春町
田川郡添田町
田川郡川崎町
田川郡大任町
田川郡赤村
田川郡福智町
京都郡苅田町
京都郡みやこ町
築上郡吉富町
築上郡上毛町
築上郡築上町


### Data arrangement

In [5]:
# one hot encoding
Fukuoka_onehot = pd.get_dummies(Fukuoka_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Fukuoka_onehot['Neighborhood'] = Fukuoka_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Fukuoka_onehot.columns[-1]] + list(Fukuoka_onehot.columns[:-1])
Fukuoka_onehot = Fukuoka_onehot[fixed_columns]

In [6]:
Fukuoka_grouped = Fukuoka_onehot.groupby('Neighborhood').mean().reset_index()
Fukuoka_grouped


,Neighborhood,Adult Boutique,American Restaurant,Arcade,Art Museum,Arts & Crafts Store,Athletics & Sports,Auto Garage,BBQ Joint,Baby Store,Bakery,Bar,Baseball Field,Baseball Stadium,Basketball Court,Bed & Breakfast,Bike Shop,Boat or Ferry,Bookstore,Bridal Shop,Bridge,Buffet,Burger Joint,Bus Station,Bus Stop,Café,Castle,Chinese Restaurant,City Hall,Clothing Store,Coffee Shop,Concert Hall,Convenience Store,Convention Center,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Donburi Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store,Farm,Farmers Market,Fast Food Restaurant,Fish Market,Fishing Store,Food & Drink Shop,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Garden Center,Gift Shop,Golf Course,Golf Driving Range,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Historic Site,History Museum,Hobby Shop,Home Service,Hot Spring,Hotel,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Kaiseki Restaurant,Karaoke Bar,Karaoke Box,Korean Restaurant,Lake,Liquor Store,Market,Men's Store,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Mountain,Moving Target,Multiplex,Museum,Nabe Restaurant,Nightclub,Noodle House,Okonomiyaki Restaurant,Outdoor Sculpture,Outdoor Supply Store,Pachinko Parlor,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Pharmacy,Pizza Place,Playground,Pub,Public Bathroom,Ramen Restaurant,Rental Service,Reservoir,Rest Area,Restaurant,River,Sake Bar,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Shopping Mall,Shopping Plaza,Skate Park,Snack Place,Soba Restaurant,South Indian Restaurant,Souvenir Shop,Spa,Sporting Goods Shop,Sri Lankan Restaurant,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Takoyaki Place,Tea Room,Tempura Restaurant,Thai Restaurant,Theater,Thrift / Vintage Store,Toll Booth,Tonkatsu Restaurant,Track Stadium,Train Station,Tree,Tunnel,Udon Restaurant,Unagi Restaurant,Used Bookstore,Video Store,Vineyard,Wagashi Place,Water Park,Wings Joint,Yakitori Restaurant,Yoshoku Restaurant,Zoo
0,うきは市,0.0,0.000000,0.000000,0.000000,0.062500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0625,0.000000,0.125000,0.000000,0.000000,0.000000,0.000000,0.062500,0.000000,0.125000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.062500,0.000000,0.000000,0.062500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.125000,0.062500,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.0625,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.062500,0.0,0.000000,0.062500,0.000000,0.000000,0.000000,0.0,0.062500,0.000000,0.000000,0.000000,0.000000,0.000000
1,みやま市,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.100000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.400000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0000

In [7]:
kita=Fukuoka_grouped[Fukuoka_grouped['Neighborhood'].str.contains('北九州市')]
kk=kita.mean(axis=0)
kk['Neighborhood']='北九州市'
list=kita['Neighborhood'].tolist()
Fukuoka_kita=Fukuoka_grouped.query('Neighborhood != list')
Fukuoka_kita.loc['61']=kk

fuku=Fukuoka_kita[Fukuoka_kita['Neighborhood'].str.contains('福岡市')]
fk=fuku.mean(axis=0)
fk['Neighborhood']='福岡市'
list=fuku['Neighborhood'].tolist()
Fukuoka_fuku=Fukuoka_kita.query('Neighborhood != list')
Fukuoka_fuku.loc['62']=fk
Fukuoka_arranging=Fukuoka_fuku.reset_index(drop=True)
Fukuoka_arranging.head()

/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


,Neighborhood,Adult Boutique,American Restaurant,Arcade,Art Museum,Arts & Crafts Store,Athletics & Sports,Auto Garage,BBQ Joint,Baby Store,Bakery,Bar,Baseball Field,Baseball Stadium,Basketball Court,Bed & Breakfast,Bike Shop,Boat or Ferry,Bookstore,Bridal Shop,Bridge,Buffet,Burger Joint,Bus Station,Bus Stop,Café,Castle,Chinese Restaurant,City Hall,Clothing Store,Coffee Shop,Concert Hall,Convenience Store,Convention Center,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Donburi Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store,Farm,Farmers Market,Fast Food Restaurant,Fish Market,Fishing Store,Food & Drink Shop,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Garden Center,Gift Shop,Golf Course,Golf Driving Range,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Historic Site,History Museum,Hobby Shop,Home Service,Hot Spring,Hotel,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Kaiseki Restaurant,Karaoke Bar,Karaoke Box,Korean Restaurant,Lake,Liquor Store,Market,Men's Store,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Mountain,Moving Target,Multiplex,Museum,Nabe Restaurant,Nightclub,Noodle House,Okonomiyaki Restaurant,Outdoor Sculpture,Outdoor Supply Store,Pachinko Parlor,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Pharmacy,Pizza Place,Playground,Pub,Public Bathroom,Ramen Restaurant,Rental Service,Reservoir,Rest Area,Restaurant,River,Sake Bar,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Shopping Mall,Shopping Plaza,Skate Park,Snack Place,Soba Restaurant,South Indian Restaurant,Souvenir Shop,Spa,Sporting Goods Shop,Sri Lankan Restaurant,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Takoyaki Place,Tea Room,Tempura Restaurant,Thai Restaurant,Theater,Thrift / Vintage Store,Toll Booth,Tonkatsu Restaurant,Track Stadium,Train Station,Tree,Tunnel,Udon Restaurant,Unagi Restaurant,Used Bookstore,Video Store,Vineyard,Wagashi Place,Water Park,Wings Joint,Yakitori Restaurant,Yoshoku Restaurant,Zoo
0,うきは市,0.0,0.0,0.000000,0.0,0.0625,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0625,0.0,0.125000,0.0,0.000000,0.0,0.000000,0.0625,0.0,0.125000,0.0,0.0,0.0,0.0,0.0,0.0,0.0625,0.0,0.000000,0.062500,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.125,0.0625,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0625,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.062500,0.0,0.0,0.062500,0.0,0.000000,0.000000,0.0,0.0625,0.0,0.0,0.0,0.0,0.0
1,みやま市,0.0,0.0,0.000000,0.0,0.0000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.1,0.000000,0.0,0.000000,0.0,0.000000,0.0000,0.0,0.400000,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.000000,0.100000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.1,0.0,0.0000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0000,0.0,0.0,0.0,0.0,0.0
2,三井郡大刀洗町,0.0,0.0,0.000000,0.0,0.0000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0000,0.0,0.375000,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.125,0.0,0.0

In [8]:
Fukuoka_1=Fukuoka_arranging['Neighborhood'].str.lstrip('鞍手郡')
Fukuoka_1rp=Fukuoka_1.replace('町','鞍手町')
Fukuoka_arranging1=Fukuoka_arranging
Fukuoka_arranging1['Neighborhood']=Fukuoka_1rp

Fukuoka_2=Fukuoka_arranging1['Neighborhood'].str.lstrip('三潴郡')
Fukuoka_2rp=Fukuoka_2.replace('町','三潴町')
Fukuoka_arranging2=Fukuoka_arranging1
Fukuoka_arranging2['Neighborhood']=Fukuoka_2rp

Fukuoka_3=Fukuoka_arranging2['Neighborhood'].str.lstrip('京都郡')
Fukuoka_3rp=Fukuoka_3.replace('町','京都町')
Fukuoka_arranging3=Fukuoka_arranging2
Fukuoka_arranging3['Neighborhood']=Fukuoka_3rp

Fukuoka_4=Fukuoka_arranging3['Neighborhood'].str.lstrip('八女郡')
Fukuoka_4rp=Fukuoka_4.replace('市','八女市')
Fukuoka_arranging4=Fukuoka_arranging3
Fukuoka_arranging4['Neighborhood']=Fukuoka_4rp

Fukuoka_5=Fukuoka_arranging4['Neighborhood'].str.lstrip('嘉穂郡')
Fukuoka_5rp=Fukuoka_5.replace('町','嘉穂町')
Fukuoka_5rp2=Fukuoka_5rp.replace('麻市','嘉麻市')
Fukuoka_arranging5=Fukuoka_arranging4
Fukuoka_arranging5['Neighborhood']=Fukuoka_5rp2

Fukuoka_6=Fukuoka_arranging5['Neighborhood'].str.lstrip('朝倉郡')
Fukuoka_6rp=Fukuoka_6.replace('市','朝倉市')
Fukuoka_arranging6=Fukuoka_arranging5
Fukuoka_arranging6['Neighborhood']=Fukuoka_6rp

Fukuoka_7=Fukuoka_arranging6['Neighborhood'].str.lstrip('田川郡')
Fukuoka_7rp=Fukuoka_7.replace('市','田川市')
Fukuoka_7rp2=Fukuoka_7rp.replace('崎町','川崎町')
Fukuoka_arranging7=Fukuoka_arranging6
Fukuoka_arranging7['Neighborhood']=Fukuoka_7rp2

Fukuoka_8=Fukuoka_arranging7['Neighborhood'].str.lstrip('筑紫野郡')
Fukuoka_8rp=Fukuoka_8.replace('市','筑紫野市')
Fukuoka_8rp2=Fukuoka_8rp.replace('前町','筑前町')
Fukuoka_8rp3=Fukuoka_8rp2.replace('後市','筑後市')
Fukuoka_arranging8=Fukuoka_arranging7
Fukuoka_arranging8['Neighborhood']=Fukuoka_8rp3

Fukuoka_9=Fukuoka_arranging8['Neighborhood'].str.lstrip('築上郡')
Fukuoka_9rp=Fukuoka_9.replace('町','築上町')
Fukuoka_9rp2=Fukuoka_9rp.replace('毛町','上毛町')
Fukuoka_arranging9=Fukuoka_arranging8
Fukuoka_arranging9['Neighborhood']=Fukuoka_9rp2

Fukuoka_10=Fukuoka_arranging9['Neighborhood'].str.lstrip('糟屋郡')
Fukuoka_10rp=Fukuoka_10.replace('町','糟屋町')
Fukuoka_arranging10=Fukuoka_arranging9
Fukuoka_arranging10['Neighborhood']=Fukuoka_10rp

Fukuoka_11=Fukuoka_arranging10['Neighborhood'].str.lstrip('遠賀郡')
Fukuoka_11rp=Fukuoka_11.replace('町','遠賀町')
Fukuoka_arranging11=Fukuoka_arranging10
Fukuoka_arranging11['Neighborhood']=Fukuoka_11rp

Fukuoka_12=Fukuoka_arranging11['Neighborhood'].str.lstrip('井郡')
Fukuoka_12rp=Fukuoka_12.replace('町','井町')
Fukuoka_arranged=Fukuoka_arranging11
Fukuoka_arranged['Neighborhood']=Fukuoka_12rp


Fukuoka_arranged

,Neighborhood,Adult Boutique,American Restaurant,Arcade,Art Museum,Arts & Crafts Store,Athletics & Sports,Auto Garage,BBQ Joint,Baby Store,Bakery,Bar,Baseball Field,Baseball Stadium,Basketball Court,Bed & Breakfast,Bike Shop,Boat or Ferry,Bookstore,Bridal Shop,Bridge,Buffet,Burger Joint,Bus Station,Bus Stop,Café,Castle,Chinese Restaurant,City Hall,Clothing Store,Coffee Shop,Concert Hall,Convenience Store,Convention Center,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Donburi Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store,Farm,Farmers Market,Fast Food Restaurant,Fish Market,Fishing Store,Food & Drink Shop,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Garden Center,Gift Shop,Golf Course,Golf Driving Range,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Historic Site,History Museum,Hobby Shop,Home Service,Hot Spring,Hotel,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Kaiseki Restaurant,Karaoke Bar,Karaoke Box,Korean Restaurant,Lake,Liquor Store,Market,Men's Store,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Mountain,Moving Target,Multiplex,Museum,Nabe Restaurant,Nightclub,Noodle House,Okonomiyaki Restaurant,Outdoor Sculpture,Outdoor Supply Store,Pachinko Parlor,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Pharmacy,Pizza Place,Playground,Pub,Public Bathroom,Ramen Restaurant,Rental Service,Reservoir,Rest Area,Restaurant,River,Sake Bar,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Shopping Mall,Shopping Plaza,Skate Park,Snack Place,Soba Restaurant,South Indian Restaurant,Souvenir Shop,Spa,Sporting Goods Shop,Sri Lankan Restaurant,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Takoyaki Place,Tea Room,Tempura Restaurant,Thai Restaurant,Theater,Thrift / Vintage Store,Toll Booth,Tonkatsu Restaurant,Track Stadium,Train Station,Tree,Tunnel,Udon Restaurant,Unagi Restaurant,Used Bookstore,Video Store,Vineyard,Wagashi Place,Water Park,Wings Joint,Yakitori Restaurant,Yoshoku Restaurant,Zoo
0,うきは市,0.0,0.000000,0.000000,0.000000,0.062500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0625,0.000000,0.125000,0.000000,0.000000,0.000000,0.000000,0.062500,0.000000,0.125000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.062500,0.000000,0.000000,0.062500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.125000,0.062500,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.0625,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.062500,0.0,0.000000,0.062500,0.000000,0.000000,0.000000,0.0,0.062500,0.000000,0.000000,0.000000,0.000000,0.000000
1,みやま市,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.100000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.400000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0000

In [9]:
num_top_venues = 5

for hood in Fukuoka_arranged['Neighborhood']:
    print("----"+hood+"----")
    temp = Fukuoka_arranged[Fukuoka_arranged['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')


----うきは市----
                 venue  freq
0                 Café  0.12
1    Convenience Store  0.12
2        Grocery Store  0.12
3            Drugstore  0.06
4  Arts & Crafts Store  0.06


----みやま市----
               venue  freq
0  Convenience Store   0.4
1       Intersection   0.2
2          Drugstore   0.1
3       Noodle House   0.1
4   Ramen Restaurant   0.1


----大刀洗町----
                 venue  freq
0    Convenience Store  0.38
1         Intersection  0.25
2          Supermarket  0.12
3   Miscellaneous Shop  0.12
4  Fried Chicken Joint  0.12


----大木町----
                           venue  freq
0              Convenience Store  0.33
1                  Train Station  0.33
2                Udon Restaurant  0.17
3                    Auto Garage  0.17
4  Paper / Office Supplies Store  0.00


----中間市----
                  venue  freq
0     Convenience Store  0.23
1            Restaurant  0.10
2  Fast Food Restaurant  0.07
3        Clothing Store  0.07
4             Drugstore  0.03


---

## $Landscape$ $data$

I got landscape data in Fukuoka through web site at Fukuoka Prefectural Government.

### Download landscape data from web pdf file

In [10]:
!pip uninstall tabula-py -y

Uninstalling tabula-py-2.1.0:
  Successfully uninstalled tabula-py-2.1.0


In [11]:
!pip install tabula-py
import tabula

  Using cached https://files.pythonhosted.org/packages/53/a4/66add528eca00398af98f181772006750019eb9f2d68c7c6fdd53ba661c5/tabula_py-2.1.0-py3-none-any.whl


In [12]:

PDF_PATH = 'https://www.pref.fukuoka.lg.jp/uploaded/attachment/109588.pdf'

import tabula
dfl=tabula.read_pdf(PDF_PATH, pages = "15")
dfl_arranging=dfl[0]

### Arranging landscape data

In [13]:
dfl_arranging.head()

,Unnamed: 0,Unnamed: 1,市町村面積,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,市街地
0,NaN,NaN,NaN,農地,NaN,NaN,森林,NaN,NaN,原野等水面・河川・水路,NaN,NaN,NaN,道路,NaN,NaN,NaN,宅地,NaN,NaN,NaN,その他,NaN
1,NaN,NaN,NaN,NaN,田,畑,NaN,国有林,民有林,NaN,水面,河川,水路,NaN,一般道路,農道,林道,NaN,住宅地,工業用地,その他宅地,NaN,NaN
2,NaN,NaN,NaN,=a+b,a,b,=c+d,c,d,=e+f+g,e,f,g,=h+i+j,h,i,j,=k+l+m,k,l,m,NaN,NaN
3,福岡県,NaN,"498,652","81,300","65,100","16,200","222,251","25,023","197,228","424 21,657","4,027","12,668","4,962","32,877","29,245","1,815","1,817","75,796","46,082","5,701","24,013","64,347","56,660"
4,福岡地域,NaN,"153,767","20,073","15,997","4,076","69,039","11,064","57,975","159 4,984","1,522","2,528",934,"11,265","10,117",451,697,"27,232","17,245",851,"9,136","21,015","26,553"


In [14]:
dfl_arranging2=dfl_arranging.rename(columns={'Unnamed: 1':'Neighborhood','市町村面積':'Total area','Unnamed: 2':'Farmland','Unnamed: 3':'Rice Field','Unnamed: 4':'Field',
                              'Unnamed: 5':'Forest','Unnamed: 9':'Pond','Unnamed: 10':'River','Unnamed: 11':'Water Way','Unnamed: 13':'Road','Unnamed: 14':'Farm Road',
                              'Unnamed: 15':'Woodland Path','Unnamed: 17':'Resident','Unnamed: 18':'Industrial Site','市街地':'Town Area'})

In [15]:
de=range(5)
for i in de:
 dfl_arranging3=dfl_arranging2.drop(de).reset_index(drop=True)
dfl_arranging3

,Unnamed: 0,Neighborhood,Total area,Farmland,Rice Field,Field,Forest,Unnamed: 6,Unnamed: 7,Unnamed: 8,Pond,River,Water Way,Unnamed: 12,Road,Farm Road,Woodland Path,Unnamed: 16,Resident,Industrial Site,Unnamed: 19,Unnamed: 20,Town Area
0,40130,福岡市,"34,346","1,809","1,520",289,"10,959","2,541","8,418","71 1,024",232,706,86,"3,482","3,358",39,85,"11,364","6,961",162,"4,241","5,637","15,435"
1,40217,筑紫野市,"8,773",903,832,71,"4,429",530,"3,899",0 218,50,125,43,625,571,6,48,"1,333",885,47,401,"1,265",989
2,40218,春日市,"1,415",23,13,10,63,0,63,0 43,37,5,1,213,213,0,0,799,577,7,215,274,"1,328"
3,40219,大野城市,"2,689",51,44,7,"1,021",16,"1,005",0 96,46,48,2,318,304,0,14,860,592,16,252,343,"1,199"
4,40220,宗像市,"11,994","2,063","1,570",493,"5,015",328,"4,687",55 376,172,107,97,899,827,44,28,"1,745","1,193",11,541,"1,841",948
5,40221,太宰府市,"2,960",140,127,13,"1,181",114,"1,067",0 56,24,27,5,292,284,0,8,734,564,9,161,557,977
6,40223,古賀市,"4,207",498,399,99,"1,377",220,"1,157",0 125,49,59,17,361,342,3,16,793,502,93,198,"1,053",606
7,40224,福津市,"5,276","1,162",870,292,"1,359",79,"1,280",0 160,82,32,46,445,441,0,4,906,670,12,224,"1,244",766
8,40228,朝倉市,"24,671","4,910","3,440","1,470","13,529","1,616","11,913","0 1,219",335,678,206,"1,268",972,168,128,"1,955","1,163",255,537,"1,790",278
9,40230,糸島市,"21,570","4,254","3,500",754,"9,796","1,090","8,706",30 641,106,317,218,"1,272","1,005",136,131,"2,022","1,366",41,615,"3,555",739


In [16]:
dfl_arranging4=dfl_arranging3.drop(['Unnamed: 0','Unnamed: 6','Unnamed: 7','Unnamed: 8','Unnamed: 12','Unnamed: 16','Unnamed: 19','Unnamed: 20'],axis=1)
dfl_arranging4.loc[4:63]

,Neighborhood,Total area,Farmland,Rice Field,Field,Forest,Pond,River,Water Way,Road,Farm Road,Woodland Path,Resident,Industrial Site,Town Area
4,宗像市,"11,994","2,063","1,570",493,"5,015",172,107,97,827,44,28,"1,193",11,948
5,太宰府市,"2,960",140,127,13,"1,181",24,27,5,284,0,8,564,9,977
6,古賀市,"4,207",498,399,99,"1,377",49,59,17,342,3,16,502,93,606
7,福津市,"5,276","1,162",870,292,"1,359",82,32,46,441,0,4,670,12,766
8,朝倉市,"24,671","4,910","3,440","1,470","13,529",335,678,206,972,168,128,"1,163",255,278
9,糸島市,"21,570","4,254","3,500",754,"9,796",106,317,218,"1,005",136,131,"1,366",41,739
10,那珂川市,"7,495",373,334,39,"5,432",170,72,13,233,5,43,394,9,552
11,宇美町,"3,021",98,84,14,"1,837",11,36,3,160,0,27,321,35,465
12,篠栗町,"3,893",224,200,24,"2,617",32,32,10,183,0,37,250,10,236
13,志免町,869,65,59,6,18,12,27,2,127,2,0,300,17,637


In [18]:
dfl_arranged=dfl_arranging4.drop([20,33,44,49]).reset_index(drop=True)
lis=[1,2,3,4,5,6,7,8,9,10,11,12,13,14]
for i in lis:
 dfl_arranged.iloc[:,i]=dfl_arranged.iloc[:,i].str.replace(',','').astype(float)/dfl_arranging5['Total area'].str.replace(',','').astype(float)


In [19]:
dfl_arranged=dfl_arranged.drop('Total area',axis=1)

In [20]:
dfl_arranged.head()

,Neighborhood,Farmland,Rice Field,Field,Forest,Pond,River,Water Way,Road,Farm Road,Woodland Path,Resident,Industrial Site,Town Area
0,福岡市,0.052670,0.044256,0.008414,0.319076,0.006755,0.020556,0.002504,0.097770,0.001136,0.002475,0.202673,0.004717,0.449397
1,筑紫野市,0.102929,0.094836,0.008093,0.504844,0.005699,0.014248,0.004901,0.065086,0.000684,0.005471,0.100878,0.005357,0.112732
2,春日市,0.016254,0.009187,0.007067,0.044523,0.026148,0.003534,0.000707,0.150530,0.000000,0.000000,0.407774,0.004947,0.938516
3,大野城市,0.018966,0.016363,0.002603,0.379695,0.017107,0.017851,0.000744,0.113053,0.000000,0.005206,0.220156,0.005950,0.445891
4,宗像市,0.172003,0.130899,0.041104,0.418126,0.014341,0.008921,0.008087,0.068951,0.003669,0.002335,0.099466,0.000917,0.079040


## $Public$ $transportation$ $data$

As this data cost a lot of money, I am going to next step at the time. If the K-means clustering dose not work well, I will back here to collect this data.

## $Combine$ $Venues$ $data$ & $Landscape$ $data$

In [21]:
dataset_kmeans=pd.merge(Fukuoka_arranged, dfl_arranged, on='Neighborhood')

In [22]:
dataset_kmeans

,Neighborhood,Adult Boutique,American Restaurant,Arcade,Art Museum,Arts & Crafts Store,Athletics & Sports,Auto Garage,BBQ Joint,Baby Store,Bakery,Bar,Baseball Field,Baseball Stadium,Basketball Court,Bed & Breakfast,Bike Shop,Boat or Ferry,Bookstore,Bridal Shop,Bridge,Buffet,Burger Joint,Bus Station,Bus Stop,Café,Castle,Chinese Restaurant,City Hall,Clothing Store,Coffee Shop,Concert Hall,Convenience Store,Convention Center,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Donburi Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store,Farm,Farmers Market,Fast Food Restaurant,Fish Market,Fishing Store,Food & Drink Shop,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Garden Center,Gift Shop,Golf Course,Golf Driving Range,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Historic Site,History Museum,Hobby Shop,Home Service,Hot Spring,Hotel,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Kaiseki Restaurant,Karaoke Bar,Karaoke Box,Korean Restaurant,Lake,Liquor Store,Market,Men's Store,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Mountain,Moving Target,Multiplex,Museum,Nabe Restaurant,Nightclub,Noodle House,Okonomiyaki Restaurant,Outdoor Sculpture,Outdoor Supply Store,Pachinko Parlor,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Pharmacy,Pizza Place,Playground,Pub,Public Bathroom,Ramen Restaurant,Rental Service,Reservoir,Rest Area,Restaurant,River_x,Sake Bar,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Shopping Mall,Shopping Plaza,Skate Park,Snack Place,Soba Restaurant,South Indian Restaurant,Souvenir Shop,Spa,Sporting Goods Shop,Sri Lankan Restaurant,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Takoyaki Place,Tea Room,Tempura Restaurant,Thai Restaurant,Theater,Thrift / Vintage Store,Toll Booth,Tonkatsu Restaurant,Track Stadium,Train Station,Tree,Tunnel,Udon Restaurant,Unagi Restaurant,Used Bookstore,Video Store,Vineyard,Wagashi Place,Water Park,Wings Joint,Yakitori Restaurant,Yoshoku Restaurant,Zoo,Farmland,Rice Field,Field,Forest,Pond,River_y,Water Way,Road,Farm Road,Woodland Path,Resident,Industrial Site,Town Area
0,うきは市,0.0,0.000000,0.000000,0.000000,0.062500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0625,0.000000,0.125000,0.000000,0.000000,0.000000,0.000000,0.062500,0.000000,0.125000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.062500,0.000000,0.000000,0.062500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.125000,0.062500,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.0625,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.062500,0.0,0.000000,0.062500,0.000000,0.000000,0.000000,0.0,0.062500,0.000000,0.000000,0.000000,0.000000,0.000000,0.229865,0.133663,0.096203,0.504512,0.013451,0.034735,0.007832,0.042823,0.004257,0.005874,0.051847,0.004086,0.000000
1,みやま市,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.100000,0.000000,0.000000,0.000000,0.000

In [23]:
dataset_kmeans.dtypes

Neighborhood                      object
Adult Boutique                   float64
American Restaurant              float64
Arcade                           float64
Art Museum                       float64
Arts & Crafts Store              float64
Athletics & Sports               float64
Auto Garage                      float64
BBQ Joint                        float64
Baby Store                       float64
Bakery                           float64
Bar                              float64
Baseball Field                   float64
Baseball Stadium                 float64
Basketball Court                 float64
Bed & Breakfast                  float64
Bike Shop                        float64
Boat or Ferry                    float64
Bookstore                        float64
Bridal Shop                      float64
Bridge                           float64
Buffet                           float64
Burger Joint                     float64
Bus Station                      float64
Bus Stop        

It was accomplished that all of the variables in data except 'Neighborhood' are float type, which means the data can be applied to modeling.